A quest to answer questions about my pizza delivery dataset with sqlite
![Domino's](http://www.pizzadelivery.org/Portals/0/ContentImages/xebra_dominos_store.jpg)

First I had to figure out how to create a database from my csv and that took a few hours to get right but I'm glad I did.

In [28]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from sqlalchemy.orm import sessionmaker

path = r'C:\Users\angelddaz\OneDrive\Documents\data_training\data\RawDelData.csv'
disk_engine = create_engine('sqlite:///data.db') # Initializes database with filename 'data.db' in current directory
connection = disk_engine.connect()

j = 0
index_start = 1

for df in pd.read_csv(path, iterator=True, encoding='utf-8'):
    df['Timestamp'] = pd.to_datetime(df['Timestamp']) # Convert to datetimes, welp this line didn't work haha #TOFIX
    df.index += index_start
    columns = ['Key', 'Date', 'mmdd', 'DayOfTheWeek', 'Area', 'Distance', 'Timestamp', 'Tip', 'OrderAmount', 'TipPercent', \
               'Housing', 'GenderOfTipper', 'Cash/Credit_Tip', 'Cash/Credit_Order', 'PersonWhoDelivered', 'Area(text)', \
               'Latitude', 'Longitude', 'month']
    j+=1
    df.to_sql('data', disk_engine, if_exists='replace')
    index_start = df.index[-1] + 1
    
connection.close()

In [11]:
#Some practice queries to see if our database creation above works
df = pd.read_sql_query('SELECT Key, Tip, Housing FROM data WHERE Housing = "Apartment" ', disk_engine)
print df.tail(), "\n"
print pd.read_sql_query('SELECT Count(*) FROM data', disk_engine)

      Key   Tip    Housing
464  1287  7.71  Apartment
465  1289  3.00  Apartment
466  1296  5.00  Apartment
467  1298  3.00  Apartment
468  1299  6.47  Apartment 

   Count(*)
0      1301


In [32]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://maxcdn.icons8.com/Share/icon/Data//database1600.png", width=300, height=300)
#AWESOME. I have the database functional and ready.

1301 is the correct number of rows in our dataset so it looks like we're all good to go in this sql quest!

I'm just going to think of questions to answer in comments and then try to work out the SQL queries. It's going to start off really basic because although I am relatively familiar with SQL server and a variety of queries, I want to learn how to crawl in this setting first.

![](http://lovenewborns.com/wp-content/uploads/2016/04/NYo.jpg)

In [53]:
#1. Find out how many deliveries Angel took
print "#1"
print pd.read_sql_query('SELECT Count(*) AS [Angel Dels] FROM data AS d1\
                        WHERE d1.PersonWhoDelivered = "Angel"', disk_engine), "\n"

#2. Find out how many deliveries Sammie took
print "#2"
print pd.read_sql_query('SELECT Count(*) AS [Sam Dels] FROM data AS d1\
                        WHERE d1.PersonWhoDelivered = "Sammie"', disk_engine), "\n"

#print pd.read_sql_query('SELECT * FROM data', disk_engine)

#1
   Angel Dels
0         712 

#2
   Sam Dels
0       589 



In [57]:
#I'm going to try using sqlite using the database I created
import sqlite3
db = sqlite3.connect('data.db')

cursor = db.cursor()
cursor.execute( '''SELECT Key, PersonWhoDelivered, Tip FROM data WHERE Housing ="Apartment" 
                AND Tip >= 10 ORDER BY PersonWhoDelivered, Tip''')
print cursor.fetchall()
db.close()

[(49, u'Angel', 10.0), (603, u'Angel', 10.0), (786, u'Angel', 10.0), (107, u'Angel', 11.56), (77, u'Sammie', 10.0), (96, u'Sammie', 10.0), (138, u'Sammie', 10.0), (533, u'Sammie', 10.0), (1096, u'Sammie', 16.6)]


The formatting is not quite what I would like but I think I like this method better, using sqlite instead of the pandas querying methods.

The most fun I've had with SQL is with relational databases. This is a single table so any kind of querys I can think of would be a little boring so I'm going to leave this notebook here. Setting up the database was a challenging problem worth tackling on it's own.